In [1]:
from risk_analysis import *

2023-01-14 05:02:19.862184: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-14 05:02:20.141954: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-01-14 05:02:21.390092: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-01-14 05:02:21.390235: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] 

safety cocerns
[delayed, snag, effectiveness, safety, injured, security, emergency, technical, issue, safety, cocerns, safety cocerns]


In [2]:
text = '''
AP photographer Jack Smith, who shot Mount St. Helens, dies | The Independent Jump to contentAsia EditionChangeUK EditionUS EditionEdición en EspañolSign up to our newslettersSubscribeLog in / Register SubscribeMenuNewsNewsUKUSWorldUK PoliticsBrexitHealthBusinessScienceSpaceNews VideosSportSportFootballFormula 1Rugby UnionCricketTennisBoxingUFCCyclingGolfSport VideosVoicesVoicesEditorialsLettersTom PeckCathy NewmanJohn RentoulMary DejevskyAndrew GriceMarie Le ConteSean O’GradyCultureCultureFilmTV & RadioMusicGamesBooksArtPhotographyTheatre & DanceCulture VideosLifestyleLifestyleShoppingTechMoneyFood & DrinkFashionLove & SexWomenHealth & FamiliesRoyal FamilyMotoringElectric Vehicles Lifestyle VideosTravelTravelUK Hotel ReviewsNews & AdviceSimon CalderUKEuropeNorth AmericaAsiaInspirationCity GuidesSustainable TravelPremiumPremiumEditorialsVoicesLong ReadsPolitics ExplainedNews AnalysisMoreBestHome & GardenTechFashion & BeautyFood & DrinkKidsBooksTravel & OutdoorsSports & FitnessDealsIndyBest VideosClimateNewsVoicesSustainable LivingExplainedClimate VideosElectric Vehicles SGITVBehind The HeadlinesOn The GroundDecomplicatedBinge Or BinMillennial LoveIndy100Crosswords & PuzzlesMost CommentedNewslettersAsk Me AnythingVirtual EventsCoursesVouchersCompare    Thank you for registeringPlease refresh the page or navigate to another page on the site to be automatically logged inPlease refresh your browser to be logged inNewsAP photographer Jack Smith, who shot Mount St. Helens, diesJack Smith, an Associated Press photographer who captured unforgettable shots of the eruption of Mount St. Helens, the Exxon-Valdez oil spill and the Olympics during his 35-year career with the news organization, has diedGene JohnsonThursday 12 January 2023 23:31            Article bookmarkedFind your bookmarks in your Independent Premium section, under my profileDon't show me this message again✕AP photographer Jack Smith, who shot Mount St. Helens, diesShow all 71/7AP photographer Jack Smith, who shot Mount St. Helens, diesAP photographer Jack Smith, who shot Mount St. Helens, diesObit Jack SmithObit Jack SmithGreg Wahl-StephensAP photographer Jack Smith, who shot Mount St. Helens, diesObit Jack SmithObit Jack SmithAP1980AP photographer Jack Smith, who shot Mount St. Helens, diesObit Jack SmithObit Jack SmithAP1989AP photographer Jack Smith, who shot Mount St. Helens, diesObit Jack SmithObit Jack SmithAP1989AP photographer Jack Smith, who shot Mount St. Helens, diesObit Jack SmithObit Jack SmithAP1988AP photographer Jack Smith, who shot Mount St. Helens, diesObit Jack SmithObit Jack SmithCopyright 2023 The Associated Press. All rights reservedAP photographer Jack Smith, who shot Mount St. Helens, diesObit Jack SmithObit Jack Smith1994 AP    For free real time breaking news alerts sent straight to your inbox sign up to our breaking news emailsSign up to our free breaking news emailsPlease enter a valid email addressPlease enter a valid email addressSIGN UPI would like to be emailed about offers, events and updates from The Independent. Read our privacy notice Jack Smith, an Associated Press photographer who captured unforgettable shots of the eruption of Mount St. Helens, the Exxon-Valdez oil spill, boxer Mike Tyson biting off part of Evander Holyfield's ear, and weeping figure skater Tonya Harding at the Olympics during a 35-year career with the news organization, has died. He was 80.Smith passed away on Jan. 4 at his home in La Mesa, California. He had cancer and had been in hospice care, said his wife, Judy Smith.“People use the word legendary way too often, but in Jack’s case it might be an understatement," said David Ake, the AP's director of photography. "He could make pictures and friends faster than anyone I have ever met. If there was a big story in the West, there would be Jack — with his huge smile, beating you to the scene and making pictures you only wish you could have made.”Smith joined the AP in Chicago as a photographer in 1966, after serving in Vietnam, and spent a decade working there and in Washington, D.C. Then, in a bid to improve coverage in Oregon and lure some of the state's newspapers away from rival United Press International, the AP made him its first staff photographer in Portland in 1977, said Steve Graham, who was the bureau's news editor at the time.Smith immediately improved the photo operation not just with his keen eye and knack for getting a definitive shot, but through his exceptional organizational skills — maintaining a stable of freelancers and developing relationships with photographers at AP member newspapers around the state, Graham said.RecommendedUkraine war’s heaviest fight rages in east - follow liveCharity boss speaks out over ‘traumatic’ encounter with royal aideSmith arranged assignments for the many out-of-town photographers who arrived when Mount St. Helens, in southwestern Washington state, began rumbling in 1980. He was among the first to capture images of the volcano when it blew on May 18 that year, and he produced indelible pictures of oil-soaked wildlife following the 1989 Exxon-Valdez oil spill in Alaska's Prince William Sound.Smith had a big personality to match his 6-foot-4 frame, and was known as “Chainsaw” for his resemblance to a stereotypical lumberjack. He was among a breed of hard-charging, competitive and sharp-elbowed wire service photographers who sought to get a good image at whatever cost and in whatever conditions, several colleagues recalled.In 1988, he traveled to Barrow, Alaska, where several California gray whales had become trapped in Arctic Ocean ice. Alaska Native whalers had chopped holes where the animals could surface and breathe in an effort to save them.Knowing he was going to be on that remote assignment for days or weeks, he persuaded the AP to let him rent a snow mobile so he could reach the icebound scene whenever he needed to, recalled Don Ryan, a former AP photographer in Portland who worked with Smith for about 25 years.Furthermore, Ryan said, Smith convinced the company to buy him a shotgun, telling his bosses he needed it for protection against “rabid snow wolves.”Smith was also a talented sports photographer, staffing several Olympics and Super Bowls for the AP. He captured a famous photo of figure skater Tonya Harding at the Winter Games in Norway in 1994, with her leg up on the judges' stand, pleading tearfully to be allowed to replace a broken lace, soon after her ex-husband and bodyguard had been implicated in an attack on fellow skater Nancy Kerrigan. Another of his famous sports shots came in 1997, when boxer Mike Tyson bit off part of Evander Holyfield's ear during a fight in Las Vegas.Smith also loved to sail, frequently taking friends out on the Columbia River, traveling to the San Juan Islands in Washington state, and maintaining a 35-foot sailboat in retirement. Among his favorite assignments was the America's Cup yacht race, Ryan said.“When you went sailing with Jack you weren't there for a pleasure cruise; you were working, pulling the ropes,” Ryan said. “He went through life that way: He was the captain, and you were the crew.” Smith was exacting when it came to managing freelancers and cranky with any who failed to come back from an assignment with what he believed would have been the key shot. But those who worked with him said his high standards made them better. “He was a taskmaster, but that's how I learned,” said Eric Risberg, who has been an AP staff photographer since 1982 and credits Smith with launching his career by hiring him as a freelancer while Risberg was in college. Greg Wahl-Stephens, a longtime freelancer for the AP who became close friends with Smith, recalled being assigned to a U.S. Olympic Committee awards ceremony in Portland in 1989. Smith wanted him to get one photo featuring the two honorees, sprinter Florence Griffith Joyner and swimmer Matt Biondi. But the two did not appear onstage together, and Wahl-Stephens failed to get the shot.“He fired me — again — on that particular occasion,” Wahl-Stephens said. “But he always let me back in whenever he fired me, and he took a guy who wanted to be (the famous French photographer Henri) Cartier-Bresson and turned me into a photojournalist.”Smith grew up in Salt Lake City and first became interested in journalism as a boy, when he had a paper route and would stop by the newsroom to perform other tasks, Judy Smith said.The couple were together for 34 years, after friends set them up on a blind date in Alaska, where Smith was covering a college basketball tournament.RecommendedJeremy Corbyn refuses to tell Robert Peston if he will stand as independent MP at next electionTrump lashes out at special counsel as first 2024 campaign event announced - liveEXPLAINER: What are special counsels and what do they do?“He just loved the excitement of the job,” she said. “He loved the travel. He liked being good at something, and he was really good at what he did.”Smith leaves behind his wife; two children, Melissa and Matthew; and a granddaughter, Alexis.More aboutJack SmithAPMike TysonTonya HardingAlaskaSeattlePortlandEvander HolyfieldBarrowPeopleCaliforniaVietnamChicagoOregonExxonArctic OceanNorwayLas VegasMatthewFrenchSalt Lake City1/1AP photographer Jack Smith, who shot Mount St. Helens, diesAP photographer Jack Smith, who shot Mount St. Helens, diesObit Jack SmithGreg Wahl-Stephens    ✕Subscribe to Independent Premium to bookmark this articleWant to bookmark your favourite articles and stories to read or reference later? Start your Independent Premium subscription today.SubscribeAlready subscribed? Log inMost PopularPopular videos                               Sponsored FeaturesVoucher CodesTravelodge Discount Code5% off all bookings - Travelodge discount codeTUI Discount CodeSave £200 on 2023 holidays with this TUI discount codeBarcelo Discount Code12% off the exclusive Torre de Madrid - Barcelo discount codeVery Discount CodeExtra 20% off selected fashion and sportswear at VeryQVC discount code£5 off first orders using this QVC discount codeIndy / CompareCompare UK Broadband DealsCompare broadband packages side by side to find the best deal for youCheap Broadband Deals 2023Compare cheap broadband deals from providers with fastest speed in your areaFibre Broadband Deals January 2023All you need to know about fibre broadbandBest Apple iPhone Deals in the UK January 2023Compare iPhone contract deals and get the best offer this JanuaryCompare Mobile Phone DealsCompare the best mobile phone deals from the top networks and brandsGet in touchContact usJobsOur ProductsSubscribeRegisterNewslettersInstall our appArchiveOther publicationsInternational editionsIndependent en EspañolIndependent ArabiaIndependent TurkishIndependent PersianIndependent UrduEvening StandardNovaya GazetaExtrasPuzzlesAll topicsVoucher codesCompareIndependent AdvertisingSyndicationWorking at The IndependentLegalCode of conduct and complaintsContributorsCookie policyPrivacy noticeUser policiesModern Slavery Act    Thank you for registeringPlease refresh the page or navigate to another page on the site to be automatically logged inPlease refresh your browser to be logged inCloseAsia EditionChangeUK EditionUS EditionEdición en EspañolSubscribeLog in / Register  Indy100Crosswords & PuzzlesMost CommentedNewslettersAsk Me AnythingVirtual EventsCoursesVouchersCompare✕Log inEmail addressPasswordEmail and password don't matchSubmitForgotten your password?New to The Independent?RegisterOr if you would prefer:SIGN IN WITH GOOGLEWant an ad-free experience?View offersThis site is protected by reCAPTCHA and the Google Privacy notice and Terms of service apply.My Independent Premium    Account details    Help centre        Logout  
'''

In [3]:
get_score_article(text, '')

/home/oneautumleaf/Desktop/DjangoProjects/CRIF_India_Hackathon/Fake_Trollers/utils/risk_analysis.py:36: UserWarning: [W008] Evaluating Token.similarity based on empty vectors.
  score = max(score, token.similarity(risk_token))


26.280701754385966